In [1]:
%load_ext autoreload
%autoreload 2

from iob_ia.utils import segment, io_utils, visualise
import os

In [2]:
path_tif = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs/small_test_image.tif'
path_tif2 = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs/small_crop_multiCh.tif'
path_vsi = 'G:/20241120_IOB_Magdalena/3D_project/20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01.vsi'

In [3]:
# Specify the channels (0-based indexing)
dapi_ch = 0
gfp_ch = 1
# channels of interest
car = 2
cralbp = 3
# channel of no interest
bf = 4

In [4]:
img, voxel_size = io_utils.read_tif(path_tif2)

In [ ]:
print(img.shape)
print(voxel_size)

In [ ]:
img_gfp = img[gfp_ch]
print(img_gfp.shape)

img2, voxel_size2 = io_utils.read_vsi(path_vsi)

print(img2.shape)
print(voxel_size2)

## Segment using cellpose

In [ ]:
if os.path.exists(io_utils.gen_out_path(path_tif2, 'cp_seg')):
    mask_cp = io_utils.read_labels(io_utils.gen_out_path(path_tif2, 'cp_seg'))
else:
    mask_cp = segment.segment_3d_cellpose(img_gfp, anisotropy=voxel_size[0]/voxel_size[1])
print(mask_cp.shape)

In [ ]:
# save labels
save_path = io_utils.gen_out_path(path_tif2, 'cp_seg')
io_utils.save_labels(mask_cp, save_path)

save_path = io_utils.gen_out_path(path_tif2, 'gfpCh')
io_utils.save_labels(img_gfp, save_path)

## Filter lable image on features
shape features: 'area, 'euler_number' and 'extent'
intensity features: "intensity_max", "intensity_mean", "intensity_min"

In [ ]:
label2remove = segment.filter_shape(mask_cp, 'area', min_val=400)
#print(lbls)

In [ ]:
label2remove = segment.filter_intensity(mask_cp, img_gfp, 'intensity_mean', min_val=4000, labels_to_remove=label2remove)
#print(lbls)

In [ ]:
nuclei = segment.remove_label_objects(mask_cp, labels=label2remove)

In [ ]:
save_path = io_utils.gen_out_path(path_tif2, 'filtered')
io_utils.save_labels(nuclei, save_path)

## Visualise

In [ ]:
#visualise.add_pair(img=img_gfp, labels=nuclei, name='image')
visualise.add_image(img=img_gfp, name='gfp', colormap='green', scale=voxel_size)
visualise.add_image(img=img[car], name='car', colormap='red', scale=voxel_size)
visualise.add_labels(nuclei, name='nuclei', scale=voxel_size)

## Label mask modulations

In [ ]:
# Create cell an cytoplasm mask from nuclei
cells, cyto = segment.create_cell_cyto_masks(nuclei, expansion=2.5, voxel_size=voxel_size)
visualise.add_labels(cells, name='cells', scale=voxel_size)
visualise.add_labels(cyto, name='cyto', scale=voxel_size)

In [ ]:
# Testing how cellpose look on the car channel...

nuc_car = segment.segment_3d_cellpose(img[car], anisotropy=voxel_size[0]/voxel_size[1])
visualise.add_labels(nuc_car, name='car_cp', scale=voxel_size)

In [24]:
# Testing custom model
import os
model_path='G:\\20241120_IOB_Magdalena\\3D_project\\20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs\\training_folder\\models\\CP_cone_cells_20250207_113638'

nuc_gfp = segment.segment_3d_cellpose(
    img[gfp_ch], anisotropy=None, #voxel_size[0]/voxel_size[1],
    model_path=model_path)
visualise.add_pair(img[gfp_ch], nuc_gfp, name='gfp', scale=voxel_size)

2025-02-07 18:19:16,895 [INFO] ** TORCH CUDA version installed and working. **
creating new log file
2025-02-07 18:19:16,897 [INFO] WRITING LOG OUTPUT TO C:\Users\Microscopy\.cellpose\run.log
2025-02-07 18:19:16,898 [INFO] 
cellpose version: 	3.1.0 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.5.0+cu118
>>> GPU activated: True
2025-02-07 18:19:16,900 [INFO] ** TORCH CUDA version installed and working. **
2025-02-07 18:19:16,901 [INFO] >>>> using GPU (CUDA)
2025-02-07 18:19:17,383 [INFO] >>>> loading model G:\20241120_IOB_Magdalena\3D_project\20240911_Hoechst 405_proA7 gfp 488_Car568_Cralbp647_BF_10x rapiclear Org2_01____TESTs\training_folder\models\CP_cone_cells_20250207_113638
2025-02-07 18:19:17,846 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-02-07 18:19:17,847 [INFO] >>>> model diam_labels =  19.667 (mean diameter of training ROIs)
2025-02-07 18:19:17,848 [INFO] multi-stack tiff read in as having 313 planes 1 channe